<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Generative_Adversarial_Networks_(GANs).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 784),
            nn.Tanh()
        )

    def forward(self, x):
        return self.fc(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(784, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

def train_gan(generator, discriminator, data_loader, num_epochs=50):
    criterion = nn.BCELoss()
    optim_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optim_d = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    for epoch in range(num_epochs):
        for real_images, _ in data_loader:
            # Train discriminator with real images
            real_labels = torch.ones(real_images.size(0), 1)
            fake_labels = torch.zeros(real_images.size(0), 1)

            optim_d.zero_grad()
            outputs = discriminator(real_images.view(-1, 784))
            loss_real = criterion(outputs, real_labels)

            # Train discriminator with fake images
            noise = torch.randn(real_images.size(0), 100)
            fake_images = generator(noise)
            outputs = discriminator(fake_images.detach())
            loss_fake = criterion(outputs, fake_labels)

            loss_d = loss_real + loss_fake
            loss_d.backward()
            optim_d.step()

            # Train generator
            optim_g.zero_grad()
            outputs = discriminator(fake_images)
            loss_g = criterion(outputs, real_labels)
            loss_g.backward()
            optim_g.step()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss D: {loss_d.item()}, Loss G: {loss_g.item()}')

# Example with random data (replace with MNIST for real-world usage)
from torch.utils.data import DataLoader, TensorDataset

data = torch.randn(1000, 784)  # Random dataset for demonstration
dataset = TensorDataset(data, data)
data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

generator = Generator()
discriminator = Discriminator()

train_gan(generator, discriminator, data_loader)